In [8]:
:load AnalysisEvents.hs
import Data.Colour
import Data.Colour.Names
import AnalysisEvents
import qualified Data.Map as M
import Types
import GHC.Exts

In [3]:
livesLL <- fmap getLfDistr (readEvents "../out/lifeExpsVal/outEvents_d0.0_r0.598.txt")
livesHH <- fmap getLfDistr (readEvents "../out/lifeExpsVal/outEvents_d2.5_r0.737.txt")

The goal here is to understand how 'typical' lifecycles change in a single location (Valencia here) between populations with low dormancy and low floral repression and populations with high dormancy and high floral repression. 
First let's start by getting the distributions for time of events (day of the year) and the length of the stages (time between events) for LL:

In [143]:
let germDLL = map (germT .> getDayYear) livesLL
let flowerDLL = map (flowerT .> getDayYear) livesLL
let ssetDLL = map (ssetT .> getDayYear) livesLL

let vegLL = map (/24) (vegSLen livesLL)
let reprLL = map (/24) (reprSLen livesLL)
let dormLL = map (/24) (dormSLen livesLL)

let lens = map (/24) (getLens livesLL)

In [144]:
plotHists "scratch/lengthsLL.png" "days" [mkHist blue lens]

![alt](scratch/lengthsLL.png "Lf lengths")

In [145]:
let hists = [mkHist' [blue] "days" t [d] | (t, d) <- zip ["germ", "flower", "seed set"] [germDLL, flowerDLL, ssetDLL]]
let lhists = [mkHist' [blue] "days" t [d] | (t, d) <- zip ["veg", "repr", "dorm"] [vegLL, reprLL, dormLL]]
plotHistsGrid "scratch/timingsLL1.png" 3 hists
plotHistsGrid "scratch/lengths.png" 3 lhists

![alt](scratch/timingsLL1.png "Timings")

![alt](scratch/lengths.png "Lengths")

In [138]:
let germDHH = map (germT .> getDayYear) livesHH
let flowerDHH = map (flowerT .> getDayYear) livesHH
let ssetDHH = map (ssetT .> getDayYear) livesHH

let vegHH = map (/24) (vegSLen livesHH)
let reprHH = map (/24) (reprSLen livesHH)
let dormHH = map (/24) (dormSLen livesHH)

let lensHH = map (/24) (getLens livesHH)

In [139]:
plotHists "scratch/lengthsHH1.png" "days" [mkHist blue lensHH]

![alt](scratch/lengthsHH1.png "Lengths")

In [140]:
let histsHH = [mkHist' [blue] "days" t [d] | (t, d) <- zip ["germ", "flower", "seed set"] [germDHH, flowerDHH, ssetDHH]]
let lhistsHH = [mkHist' [blue] "days" t [d] | (t, d) <- zip ["veg", "repr", "dorm"] [vegHH, reprHH, dormHH]]
plotHistsGrid "scratch/timingsHH.png" 3 histsHH
plotHistsGrid "scratch/lengthsSHH.png" 3 lhistsHH

![alt](scratch/timingsHH.png "Lengths")

![alt](scratch/lengthsSHH.png "Lengths")

In [109]:
plotHists "plot1.png" "day of year" [mkHist blue germDLL, mkHist red germDHH]

![alt](plot1.png "Germ timings")

In [111]:
plotHists "scratch/plot.png" "day of year" [mkHist blue flowerDLL, mkHist red flowerDHH]

![alt](scratch/plot.png "Flower timings")

In [114]:
plotHists "scratch/plot1.png" "day of year" [mkHist blue ssetDLL, mkHist red ssetDHH]

![alt](scratch/plot1.png "Sset timings")

So the timings look the same. What about the lengths?

In [115]:
plotHists "scratch/plotVegs.png" "days" [mkHist blue vegLL, mkHist red vegHH]

![alt](scratch/plotVegs.png "Veg season lengths")

In [117]:
plotHists "scratch/plotRepr.png" "days" [mkHist blue reprLL, mkHist red reprHH]

![alt](scratch/plotRepr.png "Repr season lengths")

In [119]:
plotHists "scratch/plotDorm.png" "days" [mkHist blue dormLL, mkHist red dormHH]

![alt](scratch/plotDorm.png "Seed season lengths")

So the only thing that looks different from LL to HH is the length of the seed stage. If the seed stage is longer though, why is everything else the same? How does it not affect the rest? Well, maybe the ~700 lifecycle length guys from HH get ~500 days of seed stage but because of the timing it just goes to the same time the following year, that's why the rest of the timings (day of year) are the same. 

Same thing for the LL: there must be some guys completing a lifecycle in less than a year, not as many as to cause a bimodality like the HH but still..

One way to figure that out is to sort the lifecycles into the ~400 lifecycle day guys and the ~700 day guys. Later when it comes to calculating the biomass we are not really concerned with this since the vegetative stages of the two groups looks the same. First let's just confirm the above.

In [121]:
:t sortLFs

sortLFs :: Double -> Double -> [Lifecycle] -> ([Lifecycle], [Lifecycle])

In [147]:
let (ls1, ls2) = sortLFs (400*24) (700*24) livesHH

In [151]:
let dorm1 = map (/24) (dormSLen ls1)
let dorm2 = map (/24) (dormSLen ls2)

let repr1 = map (/24) (reprSLen ls1)
    repr2 = map (/24) (reprSLen ls2)
    veg1 = map (/24) (vegSLen ls1)
    veg2 = map (/24) (vegSLen ls2)
    
let germTs1 = map (germT .> getDayYear) ls1
let germTs2 = map (germT .> getDayYear) ls2
let flowerT1 = map (flowerT .> getDayYear) ls1
let flowerT2 = map (flowerT .> getDayYear) ls2
let ssetT1 = map (ssetT .> getDayYear) ls1
let ssetT2 = map (ssetT .> getDayYear) ls2


In [153]:
plotHistsGrid "scratch/lengthsHH2.png" 3 [mkHist' [blue, red] "days" "dorm len" [dorm1, dorm2], 
                                          mkHist' [blue, red] "days" "veg len" [veg1, veg2],
                                          mkHist' [blue, red] "days" "repr len" [repr1, repr2]]

![alt](scratch/lengthsHH2.png "")

In [154]:
plotHistsGrid "scratch/timingsHH2.png" 3 [mkHist' [blue, red] "day of year" "germ time" [germTs1, germTs2], 
                                          mkHist' [blue, red] "day of year" "flower time" [flowerT1, flowerT2],
                                          mkHist' [blue, red] "day of year" "sset time" [ssetT1, ssetT2]]

![alt](scratch/timingsHH2.png "")

Although this does not affect the vegetative stage it will affect longer term metrics such as #of lifecycles in a year or the biomass in a year. So for Valencia is it better to have two groups ~400 and ~700 than to risk LL lifecycles with the possibility of ~100, which will mean very small plants -- especially with water stress (when we add it).

I will have to check this prediction with the predictions of FM. Will continue the analysis here.
I will also have to define what 'better' means.

For northern climates it's possible that the limiting factor is sustainability. So choose the only strategy that allow sustainable lifecycles, you can't afford anything else.

In [9]:
plotHistsGridR 1 [mkHist' [blue] "days" "lens" [map (/24) (getLens livesLL)]]

In [10]:
:t sortLFs

sortLFs :: Double -> Double -> [Lifecycle] -> ([Lifecycle], [Lifecycle])

In [30]:
let (lsL1, lsL2) = sortLFs (100*24) (380*24) livesLL

In [31]:
let lensl1 = map (/24) (getLens lsL1)
    lensl2 = map (/24) (getLens lsL2)

let vegl1 = map (/24) (vegSLen lsL1)
    vegl2 = map (/24) (vegSLen lsL2)
    
let germl1 = map (germT .> getDayYear) lsL1
    germl2 = map (germT .> getDayYear) lsL2

In [44]:
length lsL1
median (getLens lsL1) / 24.0
median (getLens lsL2) / 24.0
median (getLens livesLL) / 24.0

1262

140.88916806359097

350.4018512570595

329.2535252629871

In [33]:
length lsL2

4004

In [34]:
plotHistsGridR 1 [mkHist' [blue, red] "days" "lens" [lensl1, lensl2]]

In [17]:
:t mkHist'

mkHist' :: [Colour Double] -> String -> String -> [[Double]] -> Layout Double Int

In [35]:
plotHistsGridR 1 [mkHist' [blue, red] "days" "veg len" [vegl1, vegl2]]

In [39]:
avg vegl1
avg vegl2

23.3788303875518

53.677510462568065

In [40]:
plotHistsGridR 1 [mkHist' [blue, red] "day of year" "germ day" [germl1, germl2]]

In [45]:
median germl1
median germl2

200.0

272.0

### Lifecycle 1 LL (short)

Ok, so in the LL parameter regime we have two typical lifecycles. The first typical lifecycle occurs at about 20% of all the lifecycles:

In [53]:
fromIntegral (length lsL1) / fromIntegral (length lsL1 + length lsL2)

0.23965058868211167

It's a really fast lifecycle with plants germinating in the summer and completing a generation within the year.

In [55]:
median germl1
median vegl1

200.0

20.85999574015538

20th of July median germination day (200th of the year)

In [57]:
median (getLens lsL1) / 24.0

140.88916806359097

around 140 days total lifecycle length.

With 5 simulations this gives (running FMlite) a biomass of around 0.008

In [74]:
fromIntegral (length lsL1) * 0.008

10.096

### Lifecycle 2 LL (long)

In [58]:
fromIntegral (length lsL2) / fromIntegral (length lsL1 + length lsL2)

0.7603494113178884

In [63]:
median germl2
median vegl2

272.0

40.854603557182905

In [64]:
median (getLens lsL2) / 24.0

350.4018512570595

In [104]:
fromIntegral (length lsL2) * 0.0002

0.8008000000000001

In [77]:
0.008*0.2 + 0.05*0.8

4.1600000000000005e-2

### Lifecycles HH (short)

In [78]:
plotHistsGridR 1 [mkHist' [blue] "days" "lens" [map (/24) (getLens livesHH)]]

In [79]:
let (lsh1, lsh2) = sortLFs (400*24) (700*24) livesHH

In [81]:
let germts1 = map (germT .> getDayYear) lsh1
    germts2 = map (germT .> getDayYear) lsh2
    
let vegh1 = map (/24) (vegSLen lsh1)
    vegh2 = map (/24) (vegSLen lsh2)

In [84]:
let lensh1 = map (/24) (getLens lsh1)
    lensh2 = map (/24) (getLens lsh2)
    
plotHistsGridR 1 [mkHist' [blue, red] "days" "len HH" [lensh1, lensh2]]

In [82]:
plotHistsGridR 1 [mkHist' [blue, red] "days" "veg len HH" [vegh1, vegh2]]

In [83]:
plotHistsGridR 1 [mkHist' [blue, red] "day of year" "germ time" [germts1, germts2]]

In [96]:
let f1 = fromIntegral (length lsh1) / fromIntegral (length lsh1 + length lsh2)

In [97]:
let f2 = fromIntegral (length lsh2) / fromIntegral (length lsh1 + length lsh2)

In [89]:
median germts1
median vegh1

259.0

39.94946179158918

In [90]:
median germts2
median vegh2

264.0

43.70619274415852

0.1g for lifecycle 1
0.12g for lifecycle 2

In [99]:
0.00013*fromIntegral (length lsh1)

0.26428999999999997

In [100]:
0.002*fromIntegral (length lsh2)

2.2880000000000003

In [101]:
2.28 + 0.26

2.54

In [95]:
340 / 100

3.4

In [105]:
f1 * 0.1 + f2*0.12

0.10720176266918477

In [106]:
length lsh1

2033

In [107]:
length lsh2

1144

In [10]:
idlivesLL <- fmap groupById (readEvents "../out/lifeExpsVal/outEvents_d0.0_r0.598.txt")
idlivesHH <- fmap groupById (readEvents "../out/lifeExpsVal/outEvents_d2.5_r0.737.txt")

In [11]:
let nLivesL = M.elems (M.map (sortWith timeE .> dropYrsE 15 .> getLfs .> length .> fromIntegral) idlivesLL)
let nLivesH = M.elems (M.map (sortWith timeE .> dropYrsE 15 .> getLfs .> length .> fromIntegral) idlivesHH)

In [12]:
plotHistsGridR 1 [mkHist' [blue, red] "#of lives" "# of lives, HH-LL" [nLivesL, nLivesH]]